# Requirements for this program to run.
* Install the following via Anaconda Prompt
    - pyqt5 (pip install pyqt5)
    - pyqt5-tools (pip install pyqt5-tools)
    - tensorflow
    - imageai
* Trained Model and .json file (Both must be located as the same path of this file)
* Qt Designer / PyCharm
* Pandas

In [1]:
# * Install the following via Anaconda Prompt
#-------------------------------------------------
# pip install opencv-python==4.1.2.30
# pip uninstall keras-nightly -y
# pip install keras==2.3.1
# pip install tensorflow==1.14.0
# pip install 'h5py==2.10.0' --force-reinstall
# pip install imageai==2.1.5
# pip install pyqt5
# conda install -c anaconda pandas

## Surgical Face Mask Detection System Code

In [ ]:
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QDesktopWidget, QFileDialog, QMessageBox
from imageai.Detection.Custom import CustomObjectDetection
from imageai.Detection.Custom import CustomVideoObjectDetection
import os
import cv2
import sys
import csv 
import time
import csv
import argparse 
import datetime 
import pandas as pd

#-------------------------------------MAIN WINDOW--------------------------------------------------------/
class MyWindow(QMainWindow):
    def __init__(self):
        super(MyWindow, self).__init__()
        self.setWindowTitle("COVID-19 Medical Face Mask Detection")
        self.resize(1076, 700)
        self.setMinimumSize(QtCore.QSize(1076, 700))
        self.setMaximumSize(QtCore.QSize(1076, 700))
        self.setFocusPolicy(QtCore.Qt.NoFocus)
        self.setStyleSheet("background-image: url(deni/bgg.png);")
        self.initUI()
        self.center()
        
    def center(self):
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())

        
#--------------------------------------BUTTONS--------------------------------------------------------/
   
    def initUI(self):
        self.pushButton = QtWidgets.QPushButton(self)
        self.pushButton.setGeometry(QtCore.QRect(730, 170, 181, 51))
        self.pushButton.setText("Upload Image")
        self.pushButton.setMaximumSize(QtCore.QSize(1076, 713))
        self.pushButton.setStyleSheet("background-image: url(deni/nn.png);")
        self.pushButton.setFlat(True)
        self.pushButton_2 = QtWidgets.QPushButton(self)
        self.pushButton_2.setGeometry(QtCore.QRect(730, 310, 181, 51))
        self.pushButton_2.setText("Take Video")
        self.pushButton_2.setMaximumSize(QtCore.QSize(1076, 713))
        self.pushButton_2.setStyleSheet("background-image: url(deni/nn.png);")
        self.pushButton_2.setFlat(True)
        self.pushButton_3 = QtWidgets.QPushButton(self)
        self.pushButton_3.setGeometry(QtCore.QRect(730, 450, 181, 51))
        self.pushButton_3.setText("Upload Video")
        self.pushButton_3.setMaximumSize(QtCore.QSize(1076, 713))
        self.pushButton_3.setStyleSheet("background-image: url(deni/nn.png);")
        self.pushButton_3.setFlat(True)
        
        #WHEN PRESSED:
        self.pushButton.clicked.connect(self.getImage)
        self.pushButton_2.clicked.connect(self.Livestream)
        self.pushButton_3.clicked.connect(self.getVideo)
        
#------------------------------------DETECTION CODE-------------------------------------------------------------/

#------------------------------SINGLE IMAGE DETECTION-----------------------------------------------------------/      
   
    def getImage(self):
        fname = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Image files (*.jpg *.png *.jpeg)")
        #Setting Variable name "imagePath" to global so it can be called at def DetectImage()
        global imagePath
        imagePath = fname[0]
        self.DetectImage()
        
    def DetectImage(self):
        from datetime import datetime
        classid = []
        percentage =[]
        execution_path = os.getcwd()
        today = datetime.now()
        
        detector = CustomObjectDetection()
        detector.setModelTypeAsYOLOv3()
        detector.setModelPath("detection_model-ex-009--loss-0003.310.h5")
        detector.setJsonPath("detection_config.json")
        detector.loadModel()
        detections = detector.detectObjectsFromImage(input_image=imagePath, output_image_path= imagePath + "detected_facemask.jpg", minimum_percentage_probability=50)
        
        for eachObject in detections:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])
                
        #CSV FILE FROM THE IMAGE:
        with open('single-image.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupImage()
    
    #POP-UP WINDOW
    def popupImage(self):
        msg = QMessageBox()
        msg.setWindowTitle("Single-Image Detection")
        msg.setText("Single Detection Finish")
        msg.setIcon(QMessageBox.Information)
        x = msg.exec_()

#-------------------------------------VIDEO DETECTION-------------------------------------------------------/   
        
    def getVideo(self):
        vidname = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Video files (*.avi *.mp4)")
        #Setting Variable name "aviPath" to global so it can be called at def DetectImage()
        global aviPath
        aviPath = vidname[0]
        #call def DectectVideo
        self.DetectVideo()
        
    def DetectVideo(self):
        from datetime import datetime
        classid = []
        percentage =[]
        execution_path = os.getcwd()

        def forFrame(frame_number, output_array, output_count):
            for eachObject in output_array:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])
                
        rec_detector = CustomVideoObjectDetection()
        rec_detector.setModelTypeAsYOLOv3()
        rec_detector.setModelPath("detection_model-ex-009--loss-0003.310.h5")
        rec_detector.setJsonPath("detection_config.json")
        rec_detector.loadModel()
        rec_detector.detectObjectsFromVideo(input_file_path=aviPath,
                                                  output_file_path=os.path.join(execution_path, aviPath + "detected_facemask"),
                                                  per_frame_function=forFrame,
                                                  minimum_percentage_probability=30,
                                                  log_progress=True)
        
        
        #CSV FILE FROM THE VIDEO:
        with open('video-image.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupImage()
    
    #POP-UP WINDOW
    def popupVid(self):
        msg2 = QMessageBox()
        msg2.setWindowTitle("Video Detection")
        msg2.setText("Video Detection Finish")
        msg2.setIcon(QMessageBox.Information)
        y = msg2.exec_()
    
#----------------------------------LIVESTREAM DETECTION---------------------------------------------------------/
     
    
    def Livestream(self):
        from datetime import datetime
        classid = []
        percentage =[]

        execution_path = os.getcwd()
        camera = cv2.VideoCapture(0)
        
        def forFrame(frame_number, output_array, output_count):
            for eachObject in output_array:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])

        video_detector = CustomVideoObjectDetection()
        video_detector.setModelTypeAsYOLOv3()
        video_detector.setModelPath("detection_model-ex-009--loss-0003.310.h5")
        video_detector.setJsonPath("detection_config.json")
        video_detector.loadModel()
        video_detector.detectObjectsFromVideo(camera_input=camera, 
                                                                output_file_path=os.path.join(execution_path, "LiveFeedOutput001"),
                                                                per_frame_function=forFrame,  minimum_percentage_probability=50,
                                                                detection_timeout=3)


        #CSV FILE FROM THE LIVESTREAM:
        with open('live_data.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupLive()
    
    #POP-UP WINDOW
    def popupLive(self):
        msg3 = QMessageBox()
        msg3.setWindowTitle("Live Detection")
        msg3.setText("Live Dectection Finish")
        msg3.setIcon(QMessageBox.Information)
        z = msg3.exec_()
        
#-------------------------------OPEN FORM /END OF CODE/-------------------------------------------------------------/
if __name__ == "__main__":
    app = QApplication(sys.argv)
    win = MyWindow()
    win.show()
    sys.exit(app.exec_())

Using TensorFlow backend.
/opt/anaconda3/envs/session4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/session4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/session4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/session4/lib/pytho